# Ejemplo 2: Tipo de Cambio del BCRA

El procedimiento es similar al ejemplo anterior, por lo que vamos a generar una lista de fechas, ya que la página
http://www.bcra.gov.ar/PublicacionesEstadisticas/Cotizaciones_por_fecha.asp \
Nos permite extraer un día a la vez.

In [ ]:
import requests
import pandas as pd
from datetime import datetime, timedelta
import io

Generamos el listado de fechas indicando el inicio y el final del período.

In [ ]:
fecha_inicio = datetime.today() - timedelta(days=20)
fecha_fin = datetime.today() - timedelta(days=1)
fechas = pd.date_range(start=fecha_inicio, end=fecha_fin, freq='B')
fechas

Como la fecha se encuentra con la hora, es conveniente cambiar el formato de cada fecha para que quede como lo pide la página.

In [ ]:
fechas = [fecha.strftime('%d/%m/%Y') for fecha in fechas]
fechas

### Prueba con una sola fecha

In [ ]:
url= 'http://www.bcra.gov.ar/PublicacionesEstadisticas/Cotizaciones_por_fecha_2.asp'
post_data ={'pp1': '1', 'date2':fechas[3]}
response = requests.post(url, data=post_data)

Luego de recibir la respuesta, leemos la tabla con pandas. En este caso le vamos a indicar cuales son los separadores de miles y el simobolo de decimal

In [ ]:
df = pd.read_html(io.BytesIO(response.content), decimal=',', thousands='.')[0]
df

En la tabla vemos que hay un título por encima del nombre de las columnas, por lo que tenemos que verificar si tiene un índice múltiple. \
Al verificar el nombre de las columnas, vemos que cada columna esta compuesta por una tupla.\
Es conveniente que los nombres de las columnas no queden como tuplas, por lo que hay que eliminar el índice múltiple (utilizamos la función droplevel)

In [ ]:
#Verificamos si tienen multi index
df.columns

In [ ]:
#Para formatear un poco la tabla y que podamos recopilar varias (deben tener el mismo nombre de columna)
#eliminamos el multi index
#Conviene revisar como queda luego para ver que no perdimos una parte relevante del nombre de las columnas
df.columns = df.columns.droplevel()
df.columns

In [ ]:
df

Le agregamos la fecha al dataframe y verificamos como queda al final

In [ ]:
#En esta parte le vamos a agregar una columna con la fecha
df['Date'] = fechas[1]
df.set_index('Date', inplace=True)

df.head()

## Recopilación de varias fechas

Como en el ejemplo 1, vamos a iterar sobre las fechas y en cada iteración vamos a actualizar la post data.\
Luego de contar con la respuesta de la solicitud, hacemos la lectura del contenido de la página y aplicamos todas las correcciones que hicimos con una sola fecha.

En este caso, es posible que la página nos muestre una tabla pero sin filas si para la fecha elegida no hubo cotizaciones, a pesar de ser un día de semana (por ejemplo, para un feriado). Por eso se le agrega la condición que la cantidad de filas sean diferente a 0.



In [ ]:
url= 'http://www.bcra.gov.ar/PublicacionesEstadisticas/Cotizaciones_por_fecha_2.asp'

#Lo hacemos iterativo
for indice, fecha in enumerate(fechas):
    #Definimos los datos que se envían como post data
    post_data ={'pp1': '1', 'date2':fecha}
    response = requests.post(url, data=post_data)
    
    df_temp = pd.read_html(io.BytesIO(response.content), decimal=',', thousands='.')[0]
    
    #Si la cantidad de filas no es nula, realizo todos los cambios
    if df_temp.shape[0] != 0:
    
        #Verificamos si hay mult index
        if df_temp.columns.nlevels == 2:
            df_temp.columns = df_temp.columns.droplevel()

        df_temp['Date'] = fecha
        
        #Hacemos el append
        if indice == 0:
            df = df_temp.copy()
        else:
            df = df.append(df_temp, ignore_index=True)            
    
    else:
        continue

In [ ]:
df